In [1]:
%matplotlib widget

In [2]:
import backtrader as bt
import pandas as pd
import pickle
from pprint import pprint

In [3]:
class IchimokuPrizecross(bt.Strategy): 
    # Moving average parameters
    params = (('pfast',50),('pslow',200),
              ('kijun_base',26),
              ("fast_period", 12),("slow_period", 26),("signal_period", 9),)

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}') # Comment this line when running optimization

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open
        
        # Order variable will contain ongoing order details/status
        self.order = None

        # Instantiate moving averages
        self.slow_sma = bt.indicators.MovingAverageSimple(self.datas[0], 
                        period=self.params.pslow)
        self.fast_sma = bt.indicators.MovingAverageSimple(self.datas[0], 
                        period=self.params.pfast)
        self.ichimoku = bt.indicators.Ichimoku(self.datas[0], 
                        kijun=self.params.kijun_base)
        macd_ind = bt.ind.MACD(
            period_me1=self.params.fast_period,
            period_me2=self.params.slow_period,
            period_signal=self.params.signal_period,
        )
        self.macd = macd_ind.macd
        self.signal = macd_ind.signal
        self.crossover = bt.ind.CrossOver(
            self.macd, self.signal
        )  # crossover buy signal

    
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # An active Buy/Sell order has been submitted/accepted - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
#                 self.log(f'BUY EXECUTED, {order.executed.price:.2f}')
                self.log(', '.join((
                    'BUY EXECUTED',
                    f'Price: {order.executed.price:.2f}',
                    f'Cost: {order.executed.value:.2f}',
                    f'Comm: {order.executed.comm:.2f}')))
            elif order.issell():
                self.log(f'SELL EXECUTED, {order.executed.price:.2f}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        
        elif order.isclosed:
            self.log(f'PNL: GROSS {order.pnl:.2f}, NET {order.pnlcomm:.2f}')

        # Reset orders
        self.order = None
    
    def next(self):
#         print(self.ichimoku.kijun_sen[0])
#         help(self.ichimoku)
#         print(len(self.position))
        # Check for open orders
        if self.order:
#             print("hello")
            return

        # Check if we are in the market
        if not self.position:
            # We are not in the market, look for a signal to OPEN trades
            ichimoku = self.dataclose[0] > self.ichimoku.kijun_sen[0] and self.dataopen[0] < self.ichimoku.kijun_sen[0]
            sma = self.dataclose[0] > self.slow_sma[0] and self.dataopen[0] > self.slow_sma[0]
            if ichimoku and sma:
            #If the 20 SMA is above the 50 SMA
#             if self.fast_sma[0] > self.slow_sma[0] and self.fast_sma[-1] < self.slow_sma[-1]:
                self.log(f'BUY CREATE {self.dataclose[0]:2f}')
    
#                 print(self.dataclose[0], self.datas[0].close[0],
#                       self.dataclose[0] == self.datas[0].close[0],
#                       self.datas[0].open[0], len(self.datas[0]), len(self.datas[0].open))
                
                # Keep track of the created order to avoid a 2nd order
#                 self.order = self.buy(plimit=(self.data.close[0]-(self.data.close[0]*0.05)), exectype=bt.Order.StopLimit)
                self.order = self.buy()
#                 self.order = self.buy()
            #Otherwise if the 20 SMA is below the 50 SMA   
#             elif self.fast_sma[0] < self.slow_sma[0] and self.fast_sma[-1] > self.slow_sma[-1]:
#                 self.log(f'SELL CREATE {self.dataclose[0]:2f}')
#                 # Keep track of the created order to avoid a 2nd order
#                 self.order = self.sell()
        else:
            # We are already in the market, look for a signal to CLOSE trades
#             print(self.bar_executed, len(self), self.order)
            
            if self.crossover < 0 or len(self) >= (self.bar_executed + 90):
                self.log(f'CLOSE CREATE {self.dataclose[0]:2f}')
                self.order = self.close()

In [4]:
class MAcrossover(bt.Strategy): 
    # Moving average parameters
    params = (('pfast',20),('pslow',50),)

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}') # Comment this line when running optimization

    def __init__(self):
        self.dataclose = self.datas[0].close
        
		# Order variable will contain ongoing order details/status
        self.order = None

        # Instantiate moving averages
        self.slow_sma = bt.indicators.MovingAverageSimple(self.datas[0], 
                        period=self.params.pslow)
        self.fast_sma = bt.indicators.MovingAverageSimple(self.datas[0], 
                        period=self.params.pfast)
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # An active Buy/Sell order has been submitted/accepted - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
#                 self.log(f'BUY EXECUTED, {order.executed.price:.2f}')
                self.log(', '.join((
                    'BUY EXECUTED',
                    f'Price: {order.executed.price:.2f}',
                    f'Cost: {order.executed.value:.2f}',
                    f'Comm: {order.executed.comm:.2f}')))

            elif order.issell():
                self.log(f'SELL EXECUTED, {order.executed.price:.2f}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            n = [order.Canceled, order.Margin, order.Rejected]
            self.log('Order Canceled/Margin/Rejected status ={}, {}'.format(order.status, n))

        # Reset orders
        self.order = None
        
    def next(self):
        # Check for open orders
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            # We are not in the market, look for a signal to OPEN trades

            #If the 20 SMA is above the 50 SMA
            if self.fast_sma[0] > self.slow_sma[0] and self.fast_sma[-1] < self.slow_sma[-1]:
                self.log(f'BUY CREATE {self.dataclose[0]:2f}')
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
            #Otherwise if the 20 SMA is below the 50 SMA   
            elif self.fast_sma[0] < self.slow_sma[0] and self.fast_sma[-1] > self.slow_sma[-1]:
                self.log(f'SELL CREATE {self.dataclose[0]:2f}')
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
        else:
            # We are already in the market, look for a signal to CLOSE trades
            if len(self) >= (self.bar_executed + 5):
                self.log(f'CLOSE CREATE {self.dataclose[0]:2f}')
                self.order = self.close()

In [5]:
cerebro = bt.Cerebro()

In [6]:
# df = pd.read_csv(r"C:\Users\saqib\Desktop\Workspace\Trading\binance_scraper\BTCUSDT-5m-data.csv", index_col='timestamp',   parse_dates=True)
data_file = r"C:\Users\saqib\Desktop\Workspace\Trading\binance_scraper\BTCUSDT-5m-data.csv"
df  =  pd.read_csv(data_file, header=0, parse_dates=True)
df = df.tail(10000)
df = df.drop(['close_time',
       'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ], axis=1)
df['timestamp'] = pd.to_datetime(df['timestamp'], format = r"%Y/%m/%d %H:%M:%S" )
df.set_index('timestamp', inplace=True)  

data = bt.feeds.PandasData(
    dataname=df,
    timeframe=bt.TimeFrame.Seconds,
    # compression=5,
    tz='Asia/Kolkata'
)

In [7]:
cerebro.adddata(data)

In [8]:
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio', timeframe=bt.TimeFrame.Minutes, compression=30)
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='py_folio')
cerebro.addanalyzer(bt.analyzers.PositionsValue, _name='p_v')
cerebro.addanalyzer(bt.analyzers.LogReturnsRolling, _name='lrr')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='ta')

In [9]:
cerebro.addsizer(bt.sizers.PercentSizer, percents=20)

In [10]:
# cerebro.addstrategy(IchimokuPrizecross)

In [11]:
cerebro.optstrategy(IchimokuPrizecross, pfast=range(5, 20), pslow=range(50, 100), kijun_base=range(5,100) )

In [12]:
# cerebro.addstrategy(MAcrossover)

In [13]:
cerebro.broker.setcash(100000.0)

In [14]:
cerebro.broker.setcommission(commission = 0.001, leverage = 20)

In [15]:
 print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00


In [ ]:
res = cerebro.run()

In [ ]:
filename = 'res.sav'
pickle.dump(res, open(filename, 'wb'))

In [ ]:
%matplotlib inline
cerebro.plot(iplot=False)

In [ ]:
res[0].analyzers.sharpe_ratio.get_analysis()

In [ ]:
res[0].broker.getvalue()

In [ ]:
res

In [ ]:
res

In [ ]:
analysis = res[0].analyzers.ta.get_analysis()

In [ ]:
pprint(analysis)

In [ ]:
res[0].analyzers.py_folio.get_analysis()['returns']

In [ ]:
res[0].analyzers.py_folio.get_analysis()

In [ ]:
res[0].analyzers.p_v.get_analysis()

In [ ]:
res[0].analyzers.lrr.get_analysis()

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df